# Imports

In [1]:
%load_ext autoreload
%autoreload 2
from fastai import *
from fastai.text import *
from fastai.callbacks import *

# Parameters

In [2]:
p_in_tokenizer_language = 'it'
p_in_max_vocab = 60000
p_in_partial_data_pct=.25

# Paths

In [10]:
base_wikipedia_path = '/storage/Data/NLPData/Wikipedia'
wikipedia_train_file_name = 'train400M.csv'
wikipedia_valid_file_name = 'val400M.csv'
wikipedia_merged_file_name = 'merged400M.csv'
wikipedia_tokenized_pickle_file_name = 'wikipedia_100_tokenized.pkl'
wikipedia_tokenized_itos_pickle_file_name = 'wikipedia_100_tokenized_itos.pkl'

In [4]:
PATH = Path(base_wikipedia_path)

# Load data from files and merging

In [ ]:
train = pd.read_csv(PATH/wikipedia_train_file_name, header=None, names=['content'])
valid = pd.read_csv(PATH/wikipedia_valid_file_name, header=None, names=['content'])

In [ ]:
valid

In [ ]:
train['is_valid'] = False
valid['is_valid'] = True

In [ ]:
df_regroup = pd.concat([train, valid])

In [ ]:
df_regroup

In [ ]:
df_regroup.to_csv(PATH/wikipedia_merged_file_name, header=None, index=None)

# Release memory

In [5]:
del df_regroup
del train
del valid

NameError: name 'df_regroup' is not defined

# Training

In [6]:
tokenizer = Tokenizer(lang=p_in_tokenizer_language)

In [7]:
data_lm_full = (TextList.from_csv(PATH, csv_name=wikipedia_merged_file_name, cols=0, processor=[TokenizeProcessor(tokenizer=tokenizer), NumericalizeProcessor(max_vocab=p_in_max_vocab)])
            .use_partial_data(p_in_partial_data_pct, seed=42)
            #Inputs: all the text files in path
            .split_from_df(col=1)
           #We may have other temp folders that contain text files so we only keep what's in train and test
            .label_for_lm()           
           #We want to do a language model so we label accordingly
            .databunch(bs=128))

# Serialization

In [9]:
data_lm_full.save(wikipedia_tokenized_pickle_file_name)

In [11]:
pickle.dump( data_lm_full.vocab.itos, open( PATH/wikipedia_tokenized_itos_pickle_file_name, "wb" ) )

In [ ]:
PATH

In [ ]:
data_lm_full = load_data(PATH, 'full_lm_60k.pkl', bs=128)

In [ ]:

len(data_lm_full.train_ds.vocab.itos)

In [ ]:
data_lm_full.show_batch()

In [ ]:

learn = language_model_learner(data_lm_full,AWD_LSTM, pretrained=False,  callback_fns=ShowGraph)
learn.lr_find()
learn.recorder.plot(skip_start=0)

In [ ]:

learn.fit_one_cycle(10, 1e-2, moms=(0.8,0.7),callbacks=[ShowGraph(learn), SaveModelCallback(learn,monitor='accuracy',mode='max')])

In [ ]:
learn.load('bestmodel60k');

In [ ]:
TEXT = "La miglior performance di Enzo Pausini è stata presentata al Festival di Sanremo il 19 ottobre 2014"
N_WORDS = 400
N_SENTENCES = 1

In [ ]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))